In [1]:
import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

In [2]:
# Load Sentinel-1 ImageCollection
sentinel_1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2020-06-01', '2020-10-01')

# Filter the Sentinel-1 collection
vv_vh_iw = sentinel_1.filter(
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')
).filter(
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')
).filter(
    ee.Filter.eq('instrumentMode', 'IW')
)

# Separate collections for ascending and descending orbits
vv_vh_iw_asc = vv_vh_iw.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
vv_vh_iw_desc = vv_vh_iw.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

# Calculate temporal means
vh_iw_asc_mean = vv_vh_iw_asc.select('VH').mean()
vh_iw_desc_mean = vv_vh_iw_desc.select('VH').mean()
vv_iw_asc_desc_mean = vv_vh_iw_asc.merge(vv_vh_iw_desc).select('VV').mean()
vh_iw_asc_desc_mean = vv_vh_iw_asc.merge(vv_vh_iw_desc).select('VH').mean()


In [4]:
import geemap
import ee

# Initialize Earth Engine
ee.Initialize()

# Create a map object
m = geemap.Map()
m.add_basemap('SATELLITE')

# Define the bounding box for Ilam District, Nepal
ilam_bbox = ee.Geometry.Polygon(
    [[
        [87.3520, 26.5210], [87.4319, 26.3855], [88.1310, 26.4555], [87.5919, 27.1022], [87.4240, 27.0300], [87.3520, 26.5210]
    ]]
)

# Filter Sentinel-1 data for Ilam District
sentinel_1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(ilam_bbox).filterDate('2020-06-01', '2020-10-01')

# Filter the Sentinel-1 collection
vv_vh_iw = sentinel_1.filter(
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')
).filter(
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')
).filter(
    ee.Filter.eq('instrumentMode', 'IW')
)

# Separate collections for ascending and descending orbits
vv_vh_iw_asc = vv_vh_iw.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
vv_vh_iw_desc = vv_vh_iw.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

# Calculate temporal means
vh_iw_asc_mean = vv_vh_iw_asc.select('VH').mean()
vh_iw_desc_mean = vv_vh_iw_desc.select('VH').mean()
vv_iw_asc_desc_mean = vv_vh_iw_asc.merge(vv_vh_iw_desc).select('VV').mean()
vh_iw_asc_desc_mean = vv_vh_iw_asc.merge(vv_vh_iw_desc).select('VH').mean()

# Define color palette with a gradient from purple to yellow
color_palette = {'min': -18, 'max': -4, 'palette': ['gray','black','yellow']}

# Add layers to the map, clipped to Ilam District
m.add_layer(vv_iw_asc_desc_mean.clip(ilam_bbox), color_palette, 'VV Ascending/Descending Mean')
m.add_layer(vh_iw_asc_desc_mean.clip(ilam_bbox), color_palette, 'VH Ascending/Descending Mean')
m.add_layer(vh_iw_asc_mean.clip(ilam_bbox), color_palette, 'VH Ascending Mean')
m.add_layer(vh_iw_desc_mean.clip(ilam_bbox), color_palette, 'VH Descending Mean')

# Set the center and zoom level to Ilam District
m.set_center(87.5919, 26.7900, 10)  # Coordinates of the center and zoom level adjusted for Ilam District

# Display the map
m


Map(center=[26.79, 87.5919], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…